In [151]:
CurrentPortfolio

,position,symbol
0,-260.0,KEY
1,-3.0,KLAC
2,372.0,KEP
3,86.0,TOT
4,5.0,AXP
...,...,...
950,35.0,GGG
951,-674.0,PE
952,-11.0,PCTY
953,12.0,TRU


In [175]:
ib.disconnect()

In [164]:
new_amount_portfolio[0]

21.68520153542514

In [173]:
ib.disconnect()

In [174]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
#from scipy.stats.stats import pearsonr
import random
import json
from itertools import permutations
import numpy as np
import copy
import pickle
import time
import multiprocessing as mp
import math
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import datetime
import datetime as dt
import ib_insync
print(ib_insync.__all__)

from ib_insync import *
#Andy - not using Jupyter
util.startLoop()

#Andy- this is the default account, setting so that orders can be allocated to it.
acc_num = 'DU1870227'

#Connect to IB Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=11,timeout=100)



def format_days(x):
    if len(x)<2:
        return("0"+x)
    else:
        return(x)
    
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

target_position = pd.read_csv("/home/sevder/ibtest/Alphas/Alpha5/2020_6_12.csv",index_col='Date')
target_position.index = pd.to_datetime(target_position.index)
target_position = target_position[target_position.index==target_position.index[-1]]

ExchangeData = pd.read_csv("exchange.csv")
ExchangeData.index= ExchangeData["Stock"]
#Andy - this seems to be a test line
#ExchangeData.loc["AAPL"]["primaryExchange"]=="NASDAQ"
StrategyDay = str(target_position.index[0].year)+"-"+format_days(str(target_position.index[0].month))+"-"+format_days(str(target_position.index[0].day))

CurrentPortfolio = pd.DataFrame()
for element in ib.positions(account=acc_num):
    symbol = str(element).split("symbol='")[1].split("'")[0]
    position =str(element).split("position=")[1].split(",")[0]
    list_row = {"symbol":symbol, "position":position}
    
    CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)
    
# CurrentPortfolio.index = CurrentPortfolio["symbol"]
PreviousPortfolio = CurrentPortfolio.copy()

CurrentPortfolio.index = CurrentPortfolio["symbol"]

for element in target_position.columns:
    print(element,"TARGET")
    try: 
        amount_portfolio = CurrentPortfolio[CurrentPortfolio.index==element]["position"][0]
        new_amount_portfolio = target_position[target_position.index==StrategyDay][element][0]

    
    # ROUNDING
    #Andy - added try to handle NaN amount
    
        amount_portfolio = int(round(float(amount_portfolio)))
        new_amount_portfolio = int(round(float(new_amount_portfolio)))
        print("rounding done")

        # Buy / Sell / Hold
        if new_amount_portfolio>amount_portfolio:
            direction = "BUY"
            trade_amount = new_amount_portfolio - amount_portfolio
        if new_amount_portfolio<amount_portfolio:
            direction = "SELL"
            trade_amount = amount_portfolio - new_amount_portfolio
        if new_amount_portfolio==amount_portfolio:
            direction = "HOLD"
            trade_amount = 0
            print(element,"HOLD")

        if direction!="HOLD":
            #Andy - changed to element as no stock variable set
            if(element not in excluse):
                try:
                    contract = Stock(element, 'SMART', 'USD', primaryExchange=ExchangeData.loc[element]["primaryExchange"])
                    order = MarketOrder(direction, trade_amount) #,account=acc_num
                    trade = ib.placeOrder(contract, order)
                    ib.sleep(0.2)
                    print(contract)
                    print(order)
                except:
                    print("NOT EXECUTED",element)
                    
    except:
        print("NOT EXECUTED",element)

['util', 'Event', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'TagValue', 'ComboLeg', 'DeltaNeutralContract', 'ContractDetails', 'ContractDescrip

Task exception was never retrieved
future: <Task finished name='Task-148' coro=<IB.connectAsync() done, defined at /home/sevder/.local/lib/python3.8/site-packages/ib_insync/ib.py:1614> exception=TimeoutError()>
Traceback (most recent call last):
  File "/usr/lib/python3.8/asyncio/tasks.py", line 280, in __step
    result = coro.send(None)
  File "/home/sevder/.local/lib/python3.8/site-packages/ib_insync/ib.py", line 1626, in connectAsync
    await self.client.connectAsync(
  File "/home/sevder/.local/lib/python3.8/site-packages/ib_insync/client.py", line 223, in connectAsync
    await asyncio.wait_for(connect(), timeout or None)
  File "/usr/lib/python3.8/asyncio/tasks.py", line 490, in wait_for
    raise exceptions.TimeoutError()
asyncio.exceptions.TimeoutError


AAPL TARGET
rounding done
AAPL HOLD
MSFT TARGET
rounding done
MSFT HOLD
AMZN TARGET
rounding done
AMZN HOLD
GOOGL TARGET
rounding done
GOOGL HOLD
GOOG TARGET
rounding done
GOOG HOLD
FB TARGET
rounding done
FB HOLD
BABA TARGET
rounding done
BABA HOLD
V TARGET
rounding done
V HOLD
JPM TARGET
rounding done
JPM HOLD
JNJ TARGET
rounding done
JNJ HOLD
MA TARGET
rounding done
MA HOLD
WMT TARGET
rounding done
WMT HOLD
BAC TARGET
rounding done
BAC HOLD
PG TARGET
rounding done
PG HOLD
TSM TARGET
rounding done
TSM HOLD
INTC TARGET
rounding done
INTC HOLD
UNH TARGET
rounding done
UNH HOLD
T TARGET
rounding done
T HOLD
HD TARGET
rounding done
HD HOLD
XOM TARGET
rounding done
XOM HOLD
KO TARGET
rounding done
KO HOLD
DIS TARGET
rounding done
DIS HOLD
NVS TARGET
rounding done
NVS HOLD
VZ TARGET
rounding done
VZ HOLD
TM TARGET
rounding done
TM HOLD
IVV TARGET
rounding done
IVV HOLD
MRK TARGET
rounding done
MRK HOLD
CVX TARGET
rounding done
CVX HOLD
CMCSA TARGET
rounding done
CMCSA HOLD
PEP TARGET
round

In [1]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
#from scipy.stats.stats import pearsonr
import random
import json
from itertools import permutations
import numpy as np
import copy
import pickle
import time
import multiprocessing as mp
import math
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import datetime
import datetime as dt
import ib_insync
print(ib_insync.__all__)

from ib_insync import *
#Andy - not using Jupyter
util.startLoop()

#Andy- this is the default account, setting so that orders can be allocated to it.
acc_num = 'DU1870227'

#Connect to IB Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=3,timeout=100)

['util', 'Event', 'SoftDollarTier', 'PriceIncrement', 'Execution', 'CommissionReport', 'BarList', 'BarDataList', 'RealTimeBarList', 'BarData', 'RealTimeBar', 'HistogramData', 'NewsProvider', 'DepthMktDataDescription', 'ScannerSubscription', 'ScanDataList', 'FundamentalRatios', 'ExecutionFilter', 'PnL', 'PnLSingle', 'AccountValue', 'TickData', 'TickByTickAllLast', 'TickByTickBidAsk', 'TickByTickMidPoint', 'HistoricalTick', 'HistoricalTickBidAsk', 'HistoricalTickLast', 'TickAttrib', 'TickAttribBidAsk', 'TickAttribLast', 'MktDepthData', 'DOMLevel', 'TradeLogEntry', 'FamilyCode', 'SmartComponent', 'PortfolioItem', 'Position', 'Fill', 'OptionComputation', 'OptionChain', 'Dividends', 'NewsArticle', 'HistoricalNews', 'NewsTick', 'NewsBulletin', 'ConnectionStats', 'Contract', 'Stock', 'Option', 'Future', 'ContFuture', 'Forex', 'Index', 'CFD', 'Commodity', 'Bond', 'FuturesOption', 'MutualFund', 'Warrant', 'Bag', 'TagValue', 'ComboLeg', 'DeltaNeutralContract', 'ContractDetails', 'ContractDescrip

<IB connected to 127.0.0.1:7497 clientId=3>

In [2]:


def format_days(x):
    if len(x)<2:
        return("0"+x)
    else:
        return(x)
    
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 


In [11]:
target_position = pd.read_csv("/home/sevder/ibtest/Alphas/Alpha5/2020_6_12.csv",index_col='Date')
target_position.index = pd.to_datetime(target_position.index)
target_position = target_position[target_position.index==target_position.index[-1]]

In [15]:
ExchangeData = pd.read_csv("exchange.csv")
ExchangeData.index= ExchangeData["Stock"]
#Andy - this seems to be a test line
#ExchangeData.loc["AAPL"]["primaryExchange"]=="NASDAQ"
StrategyDay = str(target_position.index[0].year)+"-"+format_days(str(target_position.index[0].month))+"-"+format_days(str(target_position.index[0].day))


In [149]:
CurrentPortfolio = pd.DataFrame()
for element in ib.positions(account=acc_num):
    symbol = str(element).split("symbol='")[1].split("'")[0]
    position =str(element).split("position=")[1].split(",")[0]
    list_row = {"symbol":symbol, "position":position}
    
    CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)

In [150]:
CurrentPortfolio

,position,symbol
0,-260.0,KEY
1,-3.0,KLAC
2,372.0,KEP
3,86.0,TOT
4,5.0,AXP
...,...,...
950,35.0,GGG
951,-674.0,PE
952,-11.0,PCTY
953,12.0,TRU


In [106]:
# CurrentPortfolio.index = CurrentPortfolio["symbol"]
PreviousPortfolio = CurrentPortfolio.copy()
excluse = {"UTX","RTN","S","ZAYO","GDI","SBGL","AGN","INXN","SWP","CY", 'MLNX',\
           'TSG','WBC','MSG'}

In [107]:
for i in range(0,len(target_position.columns)):
    company = target_position.columns[i]
    if company not in list(CurrentPortfolio["symbol"]):
#         symbol = target_position.columns[i]
        position = 0
        list_row = {"symbol":company, "position":position}
#         print("NOT IN",company)     
        CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)

NOT IN AMZN
NOT IN BAC
NOT IN CMCSA
NOT IN UTX
NOT IN GS
NOT IN CAT
NOT IN AGN
NOT IN RTN
NOT IN TGT
NOT IN EQIX
NOT IN ADSK
NOT IN FDX
NOT IN NXPI
NOT IN MELI
NOT IN S
NOT IN ORLY
NOT IN FLT
NOT IN CMI
NOT IN DLTR
NOT IN VMC
NOT IN ANET
NOT IN ATUS
NOT IN IP
NOT IN SWP
NOT IN NVR
NOT IN UDR
NOT IN ALNY
NOT IN CE
NOT IN TYL
NOT IN MKTX
NOT IN TAP
NOT IN DOX
NOT IN LK
NOT IN LII
NOT IN CY
NOT IN ZAYO
NOT IN GDI
NOT IN GH
NOT IN RETA
NOT IN MSG
NOT IN SBGL
NOT IN INXN
NOT IN OC
NOT IN WBC
NOT IN TSG
NOT IN PEN
NOT IN AMED


In [148]:

for element in target_position.columns:
    print(element,"TARGET")
    amount_portfolio = CurrentPortfolio[CurrentPortfolio.index==element]["position"][0]
    new_amount_portfolio = target_position[target_position.index==StrategyDay][element][0]

    
    # ROUNDING
    #Andy - added try to handle NaN amount
    try: 
        amount_portfolio = int(round(float(amount_portfolio)))
        new_amount_portfolio = int(round(float(new_amount_portfolio)))
        print("rounding done")

        # Buy / Sell / Hold
        if new_amount_portfolio>amount_portfolio:
            direction = "BUY"
            trade_amount = new_amount_portfolio - amount_portfolio
        if new_amount_portfolio<amount_portfolio:
            direction = "SELL"
            trade_amount = amount_portfolio - new_amount_portfolio
        if new_amount_portfolio==amount_portfolio:
            direction = "HOLD"
            trade_amount = 0

        if direction!="HOLD":
            #Andy - changed to element as no stock variable set
            if(element not in excluse):
                try:
                    contract = Stock(element, 'SMART', 'USD', primaryExchange=ExchangeData.loc[element]["primaryExchange"])
                    order = MarketOrder(direction, trade_amount) #,account=acc_num
                    trade = ib.placeOrder(contract, order)
                    ib.sleep(0.2)
                    print(contract)
                    print(order)
                except:
                    print("NOT EXECUTED",element)
    except:
        print("NOT EXECUTED",element)

AAPL TARGET
rounding done
Stock(symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
MarketOrder(orderId=3, clientId=3, action='SELL', totalQuantity=10)
MSFT TARGET
rounding done
Stock(symbol='MSFT', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
MarketOrder(orderId=4, clientId=3, action='SELL', totalQuantity=2)
AMZN TARGET
rounding done
Stock(symbol='AMZN', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
MarketOrder(orderId=5, clientId=3, action='BUY', totalQuantity=1)
GOOGL TARGET
rounding done
GOOG TARGET
rounding done
Stock(symbol='GOOG', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
MarketOrder(orderId=6, clientId=3, action='BUY', totalQuantity=1)
FB TARGET
rounding done
Stock(symbol='FB', exchange='SMART', primaryExchange='NASDAQ', currency='USD')
MarketOrder(orderId=7, clientId=3, action='SELL', totalQuantity=4)
BABA TARGET
rounding done
Stock(symbol='BABA', exchange='SMART', primaryExchange='NYSE', currency='USD')
Mar

ValueError: cannot convert float NaN to integer

In [ ]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
#from scipy.stats.stats import pearsonr
import random
import json
from itertools import permutations
import numpy as np
import copy
import pickle
import time
import multiprocessing as mp
import math
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import datetime
import datetime as dt
import ib_insync
print(ib_insync.__all__)

from ib_insync import *
#Andy - not using Jupyter
util.startLoop()

#Andy- this is the default account, setting so that orders can be allocated to it.
acc_num = 'DU1870227'

#Connect to IB Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=3,timeout=100)



def format_days(x):
    if len(x)<2:
        return("0"+x)
    else:
        return(x)
    
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3 

#inut file
#Andy - No dailyorders.pickle yet. Testing with a csv from GitHub.
#target_position = pd.read_pickle("dailyorders.pickle")
target_position = pd.read_csv("/home/sevder/ibtest/Alphas/Alpha5/2020_6_11.csv",index_col='Date')
target_position.index = pd.to_datetime(target_position.index)
target_position = target_position[target_position.index==target_position.index[-1]]


##insert the location of the exchange.
ExchangeData = pd.read_csv("exchange.csv")
ExchangeData.index= ExchangeData["Stock"]
#Andy - this seems to be a test line
#ExchangeData.loc["AAPL"]["primaryExchange"]=="NASDAQ"
StrategyDay = str(target_position.index[0].year)+"-"+format_days(str(target_position.index[0].month))+"-"+format_days(str(target_position.index[0].day))


#Construct CurrentPortfolio
#Andy - this portfolio() only works for the default account, and I can't yet find how to set this.
CurrentPortfolio = pd.DataFrame()
for element in ib.positions():
    symbol = str(element).split("symbol='")[1].split("'")[0]
    position =str(element).split("position=")[1].split(",")[0]
    list_row = {"symbol":symbol, "position":position}
    
    CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)


#for company in target_position.columns:
#    if company not in CurrentPortfolio["symbol"]:
#        symbol = company
#        position = 0
#        list_row = {"symbol":symbol, "position":position}
#        
#        CurrentPortfolio = CurrentPortfolio.append(list_row, ignore_index=True)

CurrentPortfolio.index = CurrentPortfolio["symbol"]
PreviousPortfolio = CurrentPortfolio.copy()
excluse = {"UTX","RTN","S","ZAYO","GDI","SBGL","AGN","INXN","SWP","CY", 'MLNX',\
           'TSG','WBC','MSG'}


for element in target_position.columns:
    print(element,"TARGET")
    amount_portfolio = CurrentPortfolio[CurrentPortfolio.index==element]["position"] 
    new_amount_portfolio = target_position[target_position.index==StrategyDay][element]

    
    # ROUNDING
    #Andy - added try to handle NaN amount
    
    amount_portfolio = int(round(amount_portfolio))
    new_amount_portfolio = int(round(new_amount_portfolio))
    print("rounding done")
    
    # Buy / Sell / Hold
    if new_amount_portfolio>amount_portfolio:
        direction = "BUY"
        trade_amount = new_amount_portfolio - amount_portfolio
    if new_amount_portfolio<amount_portfolio:
        direction = "SELL"
        trade_amount = amount_portfolio - new_amount_portfolio
    if new_amount_portfolio==amount_portfolio:
        direction = "HOLD"
        trade_amount = 0
    
    if direction!="HOLD":
        #Andy - changed to element as no stock variable set
        if(element not in excluse):
            try:
                contract = Stock(element, 'SMART', 'USD', primaryExchange=ExchangeData.loc[element]["primaryExchange"])
                order = MarketOrder(direction, trade_amount) #,account=acc_num
                trade = ib.placeOrder(contract, order)
                ib.sleep(0.2)
                print(contract)
                print(order)
            except:
                print("NOT EXECUTED",element)